# Setup

In [2]:
%load_ext autoreload
%autoreload 2 

In [3]:
import pandas as pd
from sqlalchemy import create_engine
from datasets import load_dataset
from pydantic import BaseModel
from dotenv import load_dotenv
from loguru import logger
from datetime import timedelta
import sys
import dask.dataframe as dd
from sqlalchemy.types import Text, ARRAY, JSON
import gc

sys.path.insert(0, "..")
sys.path.insert(1, "../../")

from src.utils.data_prep import parse_dt, handle_dtypes
from src.utils.data_prep import insert_chunk_to_oltp
import os

In [4]:
_ = load_dotenv(override=True)

## Arguments

In [4]:
class Args(BaseModel):
    random_seed: int = 41
    testing: bool = False

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    transaction_table_name: str = "amz_review_rating_raw"
    metadata_table_name: str = "amz_review_metadata_raw"

    hf_dataset_name: str = "McAuley-Lab/Amazon-Reviews-2023"
    amz_rating_hf_dataset_path: str = "0core_rating_only_Electronics"    #load o-core to demo real-world problem: cold-start, sparse data
    amz_metadata_hf_dataset_path: str = "raw_meta_Electronics"

    holdout_days: int = 30

    data_persist_dir: str = None

    def init(self):
        if not self.testing:
            self.data_persist_dir = os.path.abspath("../../data_for_ai/raw")   
            os.makedirs(self.data_persist_dir, exist_ok=True)
        return self

args = Args().init()
print(args.model_dump_json(indent=2))


{
  "random_seed": 41,
  "testing": false,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "transaction_table_name": "amz_review_rating_raw",
  "metadata_table_name": "amz_review_metadata_raw",
  "hf_dataset_name": "McAuley-Lab/Amazon-Reviews-2023",
  "amz_rating_hf_dataset_path": "0core_rating_only_Electronics",
  "amz_metadata_hf_dataset_path": "raw_meta_Electronics",
  "holdout_days": 30,
  "data_persist_dir": "/media/disk/real_time_recsys/data_for_ai/raw"
}


## Load dataset

In [ ]:
amz_rating_raw = load_dataset(args.hf_dataset_name,
                              args.amz_rating_hf_dataset_path,
                              split="full",
                              trust_remote_code=True)

: 

In [ ]:
amz_rating_df = amz_rating_raw.to_pandas()
amz_rating_df.drop_duplicates(subset=[args.user_col, args.item_col], inplace=True)

logger.info(f"amz_rating_df.shape: {amz_rating_df.shape}")

In [ ]:
amz_rating_df.head(5)

,user_id,parent_asin,rating,timestamp
0,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,B01G8JO5F2,5.0,1523093017534
1,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,B07N69T6TM,1.0,1592678549731
2,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,B083NRGZMM,3.0,1658185117948
3,AGGZ357AO26RQZVRLGU4D4N52DZQ,B001OC5JKY,5.0,1290278495000
4,AG2L7H23R5LLKDKLBEF2Q3L2MVDA,B07CJYMRWM,5.0,1676601581238


In [ ]:
amz_metadata_raw = load_dataset(args.hf_dataset_name,
                                args.amz_metadata_hf_dataset_path,
                                split="full",
                                trust_remote_code=True)

In [ ]:
amz_metadata_df = amz_metadata_raw.to_pandas()
amz_metadata_df.drop_duplicates(subset=[args.item_col], inplace=True)

logger.info(f"amz_metadata_df.shape: {amz_metadata_df.shape}")

2025-03-26 21:37:45.583 | INFO     | __main__:<module>:4 - amz_metadata_df.shape: (1610012, 16)


In [ ]:
with pd.option_context('display.max_colwidth',100):
    display(amz_metadata_df.head(5))

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,All Electronics,FS-1051 FATSHARK TELEPORTER V3 HEADSET,3.5,6,[],[Teleporter V3 The “Teleporter V3” kit sets a new level of value in the FPV world with Fat Shark...,None,"{'hi_res': [None], 'large': ['https://m.media-amazon.com/images/I/41qrX56lsYL._AC_.jpg'], 'thumb...","{'title': [], 'url': [], 'user_id': []}",Fat Shark,"[Electronics, Television & Video, Video Glasses]","{""Date First Available"": ""August 2, 2014"", ""Manufacturer"": ""Fatshark""}",B00MCW7G9M,None,None,None
1,All Electronics,Ce-H22B12-S1 4Kx2K Hdmi 4Port,5.0,1,"[UPC: 662774021904, Weight: 0.600 lbs]",[HDMI In - HDMI Out],None,"{'hi_res': ['https://m.media-amazon.com/images/I/51qxU4Zd5TL._AC_SL1050_.jpg', 'https://m.media-...","{'title': [], 'url': [], 'user_id': []}",SIIG,"[Electronics, Television & Video, Accessories, Cables, HDMI Cables]","{""Product Dimensions"": ""0.83 x 4.17 x 2.05 inches"", ""Item Weight"": ""5.3 ounces"", ""Item model num...",B00YT6XQSE,None,None,None
2,Computers,Digi-Tatoo Decal Skin Compatible With MacBook Pro 13 inch (Model A2338/ A2289/ A2251) - Protecti...,4.5,246,[WARNING: Please IDENTIFY MODEL NUMBER on the bottom of your Macbook. Only fits for model A2338/...,[],19.99,"{'hi_res': ['https://m.media-amazon.com/images/I/61RPxmi+mPL._AC_SL1500_.jpg', 'https://m.media-...","{'title': ['AL 2Sides Video', 'MacBook Protective Skin', 'ARTSO 15.6"" Laptop Skin Sticker Univer...",Digi-Tatoo,"[Electronics, Computers & Accessories, Laptop Accessories, Skins & Decals, Decals]","{""Brand"": ""Digi-Tatoo"", ""Color"": ""Fresh Marble"", ""Room Type"": ""Bedroom"", ""Material"": ""Vinyl"", ""T...",B07SM135LS,None,None,None
3,AMAZON FASHION,NotoCity Compatible with Vivoactive 4 band 22mm Quick Release Silicone Bands/Garmin Darth Vader/...,4.5,233,[☛NotoCity 22mm band is designed for Vivoactive 4 / Samsung Gear S3 Classic / S3 Frontier / Gear...,[],9.99,"{'hi_res': ['https://m.media-amazon.com/images/I/51ajKKbi76L._AC_UL1000_.jpg', 'https://m.media-...","{'title': [], 'url': [], 'user_id': []}",NotoCity,"[Electronics, Wearable Technology, Clips, Arm & Wristbands]","{""Date First Available"": ""May 29, 2020"", ""Manufacturer"": ""NotoCity""}",B089CNGZCW,None,None,None
4,Cell Phones & Accessories,Motorola Droid X Essentials Combo Pack,3.8,64,"[New Droid X Essentials Combo Pack, Exclusive Package Incredible Value Worth $145!!!, Includes a...","[all Genuine High Quality Motorola Made Accessories, including Multimedia Station with HDMI tech...",14.99,"{'hi_res': [None, None, None, None, None], 'large': ['https://m.media-amazon.com/images/I/51-DXS...","{'title': [], 'url': [], 'user_id': []}",Verizon,"[Electronics, Computers & Accessories, Computer Accessories & Peripherals, Memory Cards, Micro S...","{""Product Dimensions"": ""11.6 x 6.9 x 3.1 inches"", ""Item Weight"": ""1.5 pounds"", ""Other display fe...",B004E2Z88O,None,None,None


In [ ]:
cols = ["parent_asin", "main_category", "title", "description", "price", "categories", "details"]
metadata_df = amz_metadata_df[cols]
metadata_df.head(5)

,parent_asin,main_category,title,description,price,categories,details
0,B00MCW7G9M,All Electronics,FS-1051 FATSHARK TELEPORTER V3 HEADSET,[Teleporter V3 The “Teleporter V3” kit sets a ...,None,"[Electronics, Television & Video, Video Glasses]","{""Date First Available"": ""August 2, 2014"", ""Ma..."
1,B00YT6XQSE,All Electronics,Ce-H22B12-S1 4Kx2K Hdmi 4Port,[HDMI In - HDMI Out],None,"[Electronics, Television & Video, Accessories,...","{""Product Dimensions"": ""0.83 x 4.17 x 2.05 inc..."
2,B07SM135LS,Computers,Digi-Tatoo Decal Skin Compatible With MacBook ...,[],19.99,"[Electronics, Computers & Accessories, Laptop ...","{""Brand"": ""Digi-Tatoo"", ""Color"": ""Fresh Marble..."
3,B089CNGZCW,AMAZON FASHION,NotoCity Compatible with Vivoactive 4 band 22m...,[],9.99,"[Electronics, Wearable Technology, Clips, Arm ...","{""Date First Available"": ""May 29, 2020"", ""Manu..."
4,B004E2Z88O,Cell Phones & Accessories,Motorola Droid X Essentials Combo Pack,[all Genuine High Quality Motorola Made Access...,14.99,"[Electronics, Computers & Accessories, Compute...","{""Product Dimensions"": ""11.6 x 6.9 x 3.1 inche..."


In [ ]:
amz_rating_df[args.item_col].nunique()

1609860

In [ ]:
amz_metadata_df[args.item_col].nunique()

1610012

## Get holdout in order to demo data pipeline

In [ ]:
amz_rating_df = amz_rating_df.pipe(
    parse_dt
).pipe(
    handle_dtypes
)

In [ ]:
holdout_min_date = amz_rating_df[args.timestamp_col].max() - timedelta(days=args.holdout_days)

logger.info(f"The original shape of amz_rating_df: {amz_rating_df.shape}")
amz_rating_df_full = amz_rating_df.loc[lambda df: df[args.timestamp_col] < holdout_min_date]
logger.info(f"The shape of amz_rating_df after splitted: {amz_rating_df_full.shape}")

holdout_rating_df = amz_rating_df.loc[lambda df: df[args.timestamp_col] >= holdout_min_date]
logger.info(f"The shape of holdout_rating_df: {holdout_rating_df.shape}")

assert len(amz_rating_df_full) + len(holdout_rating_df) == len(amz_rating_df), "Total length of 2 subsets should be equal to the original length"

2025-03-26 21:38:02.661 | INFO     | __main__:<module>:3 - The original shape of amz_rating_df: (43365426, 4)
2025-03-26 21:38:04.486 | INFO     | __main__:<module>:5 - The shape of amz_rating_df after splitted: (43334103, 4)
2025-03-26 21:38:04.599 | INFO     | __main__:<module>:8 - The shape of holdout_rating_df: (31323, 4)


In [ ]:
amz_metadata_df_full = metadata_df.loc[lambda df: df[args.item_col].isin(amz_rating_df_full[args.item_col].unique())]
logger.info(f"The shape of amz_metadata_df_full: {amz_metadata_df_full.shape}")

holdout_metadata_df = metadata_df.loc[lambda df: ~df[args.item_col].isin(amz_metadata_df_full[args.item_col].unique())
                                      & df[args.item_col].isin(holdout_rating_df[args.item_col].unique())]
logger.info(f"The shape of holdout_metadata_df: {holdout_metadata_df.shape}")

assert len(amz_metadata_df_full) + len(holdout_metadata_df) == amz_rating_df[args.item_col].nunique(), "Total length of 2 subsets should be equal to the original length"

2025-03-26 21:38:10.508 | INFO     | __main__:<module>:2 - The shape of amz_metadata_df_full: (1607931, 7)
2025-03-26 21:38:11.233 | INFO     | __main__:<module>:6 - The shape of holdout_metadata_df: (1929, 7)


In [ ]:
amz_rating_df_full.to_parquet(args.data_persist_dir + "/amz_raw_rating.parquet", index=False)
amz_metadata_df_full.to_parquet(args.data_persist_dir + "/amz_raw_metadata.parquet", index=False)

holdout_rating_df.to_parquet(args.data_persist_dir + "/amz_holdout_rating.parquet", index=False)
holdout_metadata_df.to_parquet(args.data_persist_dir + "/amz_holdout_metadata.parquet", index=False)

# Persit to OLTP

In [ ]:
# Read data from file
#(this version caused oom on the 11gb ram machine)
# amz_rating_df_full = pd.read_parquet(args.data_persist_dir + "/amz_raw_rating.parquet")

In [ ]:
# Dask version
# Worked with using dask, the memory usage is much lower: around 4gb ram
amz_rating_df_full = dd.read_parquet(args.data_persist_dir + "/amz_raw_rating.parquet")
amz_rating_df_full = amz_rating_df_full.compute()  # Convert to Pandas if needed


In [ ]:
amz_rating_df_full.head(5)

In [5]:
host = os.getenv("POSTGRES_HOST")
port = os.getenv("POSTGRES_PORT")
database = os.getenv("POSTGRES_DB")
user = os.getenv("POSTGRES_USER")
password = os.getenv("POSTGRES_PASSWORD")
schema = os.getenv("POSTGRES_OLTP_SCHEMA")

In [6]:
connection_string = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [7]:
from sqlalchemy import create_engine, text
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Connection successful:", result.scalar() == 1)
except Exception as e:
    print("Connection failed:", e)


Connection successful: True


In [ ]:
insert_chunk_to_oltp(amz_rating_df_full, engine, schema, args.transaction_table_name)

In [ ]:
import gc
del amz_rating_df_full
gc.collect()

In [8]:
amz_metadata_df_full = dd.read_parquet(args.data_persist_dir + "/amz_raw_metadata.parquet")
amz_metadata_df_full = amz_metadata_df_full.compute()  # Convert to Pandas if needed

In [9]:
amz_metadata_df_full.head(3)

,parent_asin,main_category,title,description,price,categories,details
0,B00MCW7G9M,All Electronics,FS-1051 FATSHARK TELEPORTER V3 HEADSET,[Teleporter V3 The “Teleporter V3” kit sets a ...,None,"[Electronics, Television & Video, Video Glasses]","{""Date First Available"": ""August 2, 2014"", ""Ma..."
1,B00YT6XQSE,All Electronics,Ce-H22B12-S1 4Kx2K Hdmi 4Port,[HDMI In - HDMI Out],None,"[Electronics, Television & Video, Accessories,...","{""Product Dimensions"": ""0.83 x 4.17 x 2.05 inc..."
2,B07SM135LS,Computers,Digi-Tatoo Decal Skin Compatible With MacBook ...,[],19.99,"[Electronics, Computers & Accessories, Laptop ...","{""Brand"": ""Digi-Tatoo"", ""Color"": ""Fresh Marble..."


In [10]:
dtype = {
    "parent_asin": Text,
    "main_category": Text,
    "title": Text,
    "description": ARRAY(Text),
    "price": Text,
    "categories": ARRAY(Text),
    "details": JSON
}

In [11]:
insert_chunk_to_oltp(amz_metadata_df_full, engine, schema, args.metadata_table_name, dtype=dtype)

Ingesting chunks:   0%|          | 0/1608 [00:00<?, ?it/s]

In [12]:
# Fetch data from OLTP
with engine.connect() as connection:
    result = connection.execute(text(f"SELECT * FROM {schema}.amz_review_metadata_raw LIMIT 10"))
    test_df_hehe = pd.DataFrame(result.fetchall(), columns=result.keys())

In [13]:
test_df_hehe.head(3)

,parent_asin,main_category,title,description,price,categories,details
0,B0176741HM,Computers,GIGABYTE Video Card Graphics Cards GV-R545-1GI...,[GIGABYTE AMD Radeon HD 5450 1GB DDR3 VGA/DVI/...,128.86,"[Electronics, Computers & Accessories, Compute...","{'Max Screen Resolution': '2560 x 1600', 'Memo..."
1,B086BXKMB5,All Electronics,"hudiemm0B HDD PCI Bracket, PCI Slot 2.5inch HD...",[Specifications: Simply attach your existing 2...,0.93,"[Electronics, Computers & Accessories, Network...","{'Brand Name': 'hudiemm0B', 'Color': 'Multi', ..."
2,B09KP2195S,Computers,HP [Windows 11 Home] 2021 Newest Slim Desktop ...,"[Key Features and Benefits:, CPU, :, Intel Cel...",None,"[Electronics, Computers & Accessories, Compute...","{'Processor': '3.5 GHz celeron', 'RAM': '16 GB..."


In [ ]:
del amz_metadata_df_full
del test_df_hehe
gc.collect()

1064